# Global Solution 1 - 2TIAR - AI for RPA

## Integrantes:

*André Gomes Monteiro      RM: 89168

*Larissa Dias Cardomingo RM: 88842

*Luara Maria Marino RM: 89375

## Investimentos e conversor de moedas
De uns anos pra cá, um assunto muito discutido nas redes sociais é investimento. Nesse universo, existem centenas de possibilidades, desde aquelas para públicos que preferem aplicações sem muitos riscos, com baixa rentabilidade, até aquelas que têm mais riscos, mas, em contrapartida, possuem rentabilidade maior. 

Outra aplicação que tem se tornado interessante, é a compra de moedas de outros países.

Seu trabalho aqui, é desenvolver um software que traga informações sobre determinado investimento e conversões de moeda. Escolha 2 investimentos (por exemplo, Tesouro Direto Selic e ETF IVVB11), capture informações do comportamento desses investimentos em 2021 e plot um gráfico de linha, em que o eixo x são os meses e o eixo y, a rentabilidade. Os dois investimentos devem estar no mesmo gráfico. Em um segundo momento, desenvolva uma api, com dois endpoints de POST, em que em um endpoint, será convertido um valor enviado em real para uma moeda da sua escolha e em outro endpoint, será convertido um valor enviado em real para outra moeda da sua escolha.

Lembrem-se de deixar o código robusto. Abusem de logs, testes unitários e outros métodos vistos em aula. 


## Instalando dependencias e importando bibliotecas

In [128]:
%%capture
!pip install plotly
!pip install nbformat 
!pip install --upgrade nbformat
!pip install flask

In [129]:
import plotly.express as px
import requests
import json
from flask import Flask, request, render_template
import pandas as pd
import datetime
import plotly.graph_objects as go
import plotly
import logging
from importlib import reload 
reload(logging)

<module 'logging' from 'd:\\anaconda3\\lib\\logging\\__init__.py'>

## Criando log para segur passos do usuario

In [130]:
logging.basicConfig(filename='app.log', filemode='a',level=0, format='%(levelname)s - %(message)s')
logging.info('--- Inicio da aplicacao ---')

## Função do conversor de moeda

In [131]:
def currencyConverter(currTarget, currOrigin, value):
  # Criando variaveis auxiliares
  convertKey = currTarget + '-' + currOrigin
  convertTag = str(currTarget+currOrigin)
  
  # Retornando dados da API
  response_API = requests.get('https://economia.awesomeapi.com.br/all/{}'.format(convertKey))

  # Transformando dados em JSON de acordo com a opção de conversão escolhida	
  try:
    currentValue = response_API.json()[convertTag]['bid']
  except:
    currentValue = response_API.json()[currTarget]['bid']

  currentValue = float(currentValue)

  logging.info('Usuario fez conversao de {} para {}'.format(currOrigin, currTarget))
  return(value * currentValue)

## Função para criar o grafico de investimento

In [132]:
def getInvestmentData():
  #Criando variaveis auxiliares
  currencies = ['solana'] + ['binance-coin'] + ['ethereum'] + ['bitcoin']
  i = 0
  fig = go.Figure()

  #Coletando dados via API
  for currency in currencies:

    #Retornando dados da moeda
    url = "https://api.coincap.io/v2/assets/{}/history?interval=d1&start=1609470000000&end=1640919600000".format(currency)    
    payload = {}    
    headers= {}    
    response = requests.request("GET", url, headers=headers, data = payload)
    json_data = json.loads(response.text.encode('utf8'))
    groups = json_data["data"]

    #Retornando tag da moeda
    tag_url = "https://api.coincap.io/v2/assets/{}".format(currency)
    tag_payload = {}
    tag_headers= {}
    tag_response = requests.request("GET", tag_url, headers=tag_headers, data = tag_payload)
    tag_json_data = json.loads(tag_response.text.encode('utf8'))
    curr_tag = tag_json_data["data"]["symbol"] + " - {}".format(currency.capitalize())
    
    #Criando um dataframe para moeda e tratando dados
    df = pd.DataFrame(groups)
    df = df.drop(columns=['time'])
    df['priceUsd'] = df['priceUsd'].astype(float)
    df['priceUsd'] = df['priceUsd'].round(2)
    df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')
    df['date'] = df['date'].dt.date

    #Definindo a visibilidade no grafico
    vis = True if i<2 else 'legendonly'

    #Adicionando ao grafico
    fig.add_trace(go.Scatter(
      x=df['date'],
      y=df['priceUsd'],
      visible=vis,
      name=curr_tag
    ))
    i+=1

  logging.info('Grafico gerado com sucesso')
  #Retornando um Json do grafico
  return(json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder))

## API

#### Corpo base do html

In [144]:
html_base = '''
<!DOCTYPE html>
      <html>
      <head>
        <meta charset="UTF-8" />
        <meta http-equiv="X-UA-Compatible" content="IE=edge" />
        <meta name="viewport" content="width=device-width, initial-scale=1.0" />
        <link rel="stylesheet" href="/style.css">
        <title>Seus Investimentos</title>
        <script src="https://cdn.plot.ly/plotly-2.12.1.min.js"></script>
      </head>
      <body style="margin: 2rem">   
        <h1>Seus Investimentos</h1>     
        <div id="chart"></div>
        <figcaption style="padding-bottom: 2rem; text-align: center">Clique no nome da moeda para ver seu historico no gráfico(Valor em Dolar)</figcaption>     

        <div id="content" style= "align-items: center; display: flex; justify-content: center; flex-direction: column">
          <h2>Conversor de moeda</h2>
'''

#### Core API

In [145]:
app = Flask(__name__)

#Chamando Json do grafico
graphJSON = getInvestmentData()
  
# Metodo Get
# Responsavel por retornar o html_base com o grafico e o formulario de conversor de moedas
@app.route("/investimento", methods=["GET"])
def investimento():  
    logging.info('Pagina investimento carregada')
    return '''
      {}
          <label for="curr" style="margin: 0.5rem;">Escolha a moeda</label>
          <select name="currList" form="currConv">
            <option value="USD">Dólar</option>
            <option value="EUR">Euro</option>
            <option value="GBP">Libra</option>
            <option value="JPY">Iene</option>
            <option value="CAD">Dólar Canadense</option>
            <option value="AUD">Dólar Australiano</option>
            <option value="CHF">Franco Suiço</option>
          </select>

          <form action="/investimento/curr_converter" method="POST" id="currConv" style="display: flex; flex-direction: column; margin: 0.5rem;">
            <input type="text" name="value" placeholder="Valor" style="margin: 0.5rem;"/>
            <input type="submit" name="submit_button" value="Converter de real" style="margin: 0.5rem;"/>
            <input type="submit" name="submit_button" value="Converter para real" style="margin: 0.5rem;"/>
          </form>
        </div>
      </body>
      <script>
        var data = {};
        Plotly.newPlot("chart", data)
      </script>
    </html>
    '''.format(html_base, graphJSON)

#Metodo POST
#Recebe o valor e a moeda e retorna o valor convertido ou erro 
@app.route("/investimento/curr_converter", methods=["POST"])
def converter():
    logging.info('Endpoint de conversor de moeda chamado')
    try:
        if request.form['submit_button'] == 'Converter de real':            
            curr_converted = currencyConverter('BRL', request.form['currList'], float(request.form['value']))

            return '''
            {}
                <h3>Valor convertido: {} {:.2f}</h3>
                <button onclick="history.back()">Voltar</button>
            </div>
            </body>
            <script>
              var data = {};
              Plotly.newPlot("chart", data)
            </script>
          </html>
            '''.format(html_base, request.form['currList'], curr_converted, graphJSON)

        elif request.form['submit_button'] == 'Converter para real':
            curr_converted = currencyConverter(request.form['currList'], 'BRL', float(request.form['value']))

            return '''
            {}
                  <h3>Valor convertido: {} {:.2f}</h3>
                  <button onclick="history.back()">Voltar</button>
              </div>
              </body>
              <script>
                var data = {};
                Plotly.newPlot("chart", data)
              </script>
            </html>
            '''.format(html_base, 'BRL', curr_converted, graphJSON)

    except:
        logging.warning('Usuario inseriu valores invalidos no conversor de moedas')
        return '''
        {}
            <h3>Valor inválido!</h3>
            <button onclick="history.back()">Voltar</button>
          </div>
        </body>
          <script>
            var data = {};
            Plotly.newPlot("chart", data)
          </script>
        </html>
        '''.format(html_base, graphJSON)

  
if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5051)

127.0.0.1 - - [08/Jun/2022 18:18:24] "GET /investimento HTTP/1.1" 200 -


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5051/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Jun/2022 18:18:26] "GET /style.css HTTP/1.1" 404 -
